## Feature Engineering 

In [ ]:
# Importing relevant libraries
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, cross_val_score
import lightgbm as lgb
import optuna
import joblib

In [ ]:
# Improting cleaned datasets
historical = pd.read_csv("./cleaned_data/cleaned_historical_transactions.csv")
new_transactions = pd.read_csv("./cleaned_data/cleaned_new_merchant_transactions.csv")

In [3]:
historical.shape

(29112361, 14)

In [4]:
new_transactions.shape

(1963031, 14)

In [ ]:
# Combining the datasets
transactions = pd.concat([historical, new_transactions], ignore_index=True)
print("Combined transactions shape:", transactions.shape)

Combined transactions shape: (31075392, 14)


In [6]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31075392 entries, 0 to 31075391
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       int64  
 1   card_id               object 
 2   city_id               int64  
 3   category_1            int64  
 4   installments          int64  
 5   category_3            int64  
 6   merchant_category_id  int64  
 7   merchant_id           object 
 8   month_lag             int64  
 9   purchase_amount       float64
 10  purchase_date         object 
 11  category_2            int64  
 12  state_id              int64  
 13  subsector_id          int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 3.2+ GB


In [7]:
transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,1,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1,16,37
1,1,C_ID_4e6213e9bc,88,0,0,1,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1,16,16
2,1,C_ID_4e6213e9bc,88,0,0,1,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1,16,37
3,1,C_ID_4e6213e9bc,88,0,0,1,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1,16,34
4,1,C_ID_4e6213e9bc,88,0,0,1,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1,16,37


In [ ]:
# 1. Purchase Amount Statistics
# Computed sum, mean, standard deviation, min, max, and median of purchase amounts per customer.
purchase_stats = transactions.groupby('card_id')['purchase_amount'].agg(
    total_amount='sum',
    mean_amount='mean',
).reset_index()

In [10]:
purchase_stats

,card_id,total_amount,mean_amount
0,C_ID_00007093c1,-78.173565,-0.517706
1,C_ID_0001238066,-87.297257,-0.585888
2,C_ID_0001506ef0,-36.049233,-0.530136
3,C_ID_0001793786,-37.015633,-0.149861
4,C_ID_000183fdda,-75.428717,-0.486637
...,...,...,...
325535,C_ID_ffff1d9928,-9.938024,-0.621127
325536,C_ID_ffff579d3a,-80.957030,-0.703974
325537,C_ID_ffff756266,-5.127842,-0.205114
325538,C_ID_ffff828181,-125.063404,-0.631633


In [11]:
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

In [12]:
# 2. Transaction Frequency and Recency
# Computed the number of transactions per customer.
trans_freq = transactions.groupby('card_id').agg(
    transaction_count=('purchase_date', 'count'),
    last_purchase_date=('purchase_date', 'max')
).reset_index()

In [13]:
trans_freq.describe()

,transaction_count,last_purchase_date
count,325540.000000,325540
mean,95.457984,2018-03-04 17:08:29.012655616
min,2.000000,2017-02-01 08:06:23
25%,31.000000,2018-02-18 19:24:51
50%,60.000000,2018-04-03 17:51:03.500000
75%,118.000000,2018-04-23 17:23:45
max,5582.000000,2018-04-30 23:59:59
std,107.564639,NaN


In [14]:
# Computed recency as the difference in days between the last transaction for that customer and the max date (latest date) in the dataset as the reference.
reference_date = transactions['purchase_date'].max()
trans_freq['recency_days'] = (reference_date - trans_freq['last_purchase_date']).dt.days

In [15]:
# 3. Authorization Patterns
# Computed no. of authorized transactions for each card_id
if 'authorized_flag' in transactions.columns:
    auth_stats = transactions.groupby('card_id')['authorized_flag'].agg(
        authorized_count=lambda x: (x == 1).sum(),  
        not_authorized_count=lambda x: (x == 0).sum()
    ).reset_index()
else:
    auth_stats = pd.DataFrame()


In [16]:
auth_stats.describe()

,authorized_count,not_authorized_count
count,325540.000000,325540.000000
mean,87.726494,7.731489
std,102.167615,11.916291
min,2.000000,0.000000
25%,27.000000,1.000000
50%,54.000000,4.000000
75%,108.000000,10.000000
max,4122.000000,1510.000000


In [20]:
# 4. Installment Usage Patterns
# Computed statistics for installments and no. of transactions on installment
if 'installments' in transactions.columns:
    installments_stats = transactions.groupby('card_id')['installments'].agg(
        mean_installments='mean',
        #min_installments='min',
        #max_installments='max'
    ).reset_index()
    installment_counts = transactions[transactions['installments'] > 1].groupby('card_id').size().reset_index(name='installment_transactions')
else:
    installments_stats = pd.DataFrame()
    installment_counts = pd.DataFrame()

In [21]:
installments_stats

,card_id,mean_installments
0,C_ID_00007093c1,1.284768
1,C_ID_0001238066,1.610738
2,C_ID_0001506ef0,0.014706
3,C_ID_0001793786,0.020243
4,C_ID_000183fdda,1.806452
...,...,...
325535,C_ID_ffff1d9928,1.687500
325536,C_ID_ffff579d3a,0.026087
325537,C_ID_ffff756266,3.160000
325538,C_ID_ffff828181,1.202020


In [22]:
installment_counts.describe()

,installment_transactions
count,167181.000000
mean,11.924920
std,12.844796
min,1.000000
25%,4.000000
50%,8.000000
75%,15.000000
max,518.000000


In [23]:
# 5. Day/Time Preferences
# Extracted day, weekday, and hour from the transaction date.
transactions['day'] = transactions['purchase_date'].dt.day
transactions['weekday'] = transactions['purchase_date'].dt.weekday  # Monday=0, Sunday=6
transactions['hour'] = transactions['purchase_date'].dt.hour

# Aggregated day/time features per customer -- mean and standard deviation to capture preference and variability)
day_time_stats = transactions.groupby('card_id').agg(
    day_mean=('day', 'mean'),
    day_std=('day', 'std'),
    weekday_mean=('weekday', 'mean'),
    weekday_std=('weekday', 'std'),
    hour_mean=('hour', 'mean'),
    hour_std=('hour', 'std')
).reset_index()

In [24]:
day_time_stats

,card_id,day_mean,day_std,weekday_mean,weekday_std,hour_mean,hour_std
0,C_ID_00007093c1,13.152318,9.066237,2.218543,1.868668,14.403974,4.209796
1,C_ID_0001238066,16.503356,8.621645,3.597315,1.895655,14.818792,5.862217
2,C_ID_0001506ef0,12.205882,7.616868,3.573529,1.773067,12.691176,3.342433
3,C_ID_0001793786,17.056680,8.453894,2.805668,1.739863,15.080972,5.103156
4,C_ID_000183fdda,13.870968,9.262205,2.903226,1.818999,16.393548,5.408623
...,...,...,...,...,...,...,...
325535,C_ID_ffff1d9928,17.812500,8.320407,2.375000,2.247221,12.500000,4.926121
325536,C_ID_ffff579d3a,16.565217,7.391473,2.391304,1.940771,15.069565,4.522307
325537,C_ID_ffff756266,19.240000,9.368742,2.680000,1.375984,14.680000,5.359726
325538,C_ID_ffff828181,17.813131,7.415201,2.959596,1.824365,13.904040,4.659566


In [ ]:
# 5: Merging
features = purchase_stats.merge(trans_freq[['card_id', 'transaction_count', 'recency_days']], on='card_id', how='left')
features = features.merge(day_time_stats, on='card_id', how='left')

if not installments_stats.empty:
    features = features.merge(installments_stats, on='card_id', how='left')
if not installment_counts.empty:
    features = features.merge(installment_counts, on='card_id', how='left')

if not auth_stats.empty:
    features = features.merge(auth_stats, on='card_id', how='left')

features.fillna(0, inplace=True)

print("Feature DataFrame preview:")
print(features.head())

Feature DataFrame preview:
           card_id  total_amount  mean_amount  transaction_count  \
0  C_ID_00007093c1    -78.173565    -0.517706                151   
1  C_ID_0001238066    -87.297257    -0.585888                149   
2  C_ID_0001506ef0    -36.049233    -0.530136                 68   
3  C_ID_0001793786    -37.015633    -0.149861                247   
4  C_ID_000183fdda    -75.428717    -0.486637                155   

   recency_days   day_mean   day_std  weekday_mean  weekday_std  hour_mean  \
0            21  13.152318  9.066237      2.218543     1.868668  14.403974   
1             0  16.503356  8.621645      3.597315     1.895655  14.818792   
2            39  12.205882  7.616868      3.573529     1.773067  12.691176   
3           120  17.056680  8.453894      2.805668     1.739863  15.080972   
4             0  13.870968  9.262205      2.903226     1.818999  16.393548   

   ...  category_1_mode  category_1_mode_freq  category_1_mode_ratio  \
0  ...              0.0

In [ ]:
# Saving engineered dataset
features.to_csv("features.csv", index=False)

## End of Feature Engineering

---

---
## Implementing Models 

Prepping train, test data

In [2]:
train = pd.read_csv("./cleaned_data/train.csv")
test = pd.read_csv("./cleaned_data/test.csv")
features = pd.read_csv("features.csv")

In [3]:
train.shape

(201917, 6)

In [ ]:
# Compute the intersection of card_ids between the two datasets
common_card_ids = set(features["card_id"]).intersection(test["card_id"])

# Print out the results
print("Number of common card_ids:", len(common_card_ids))
if common_card_ids:
    print("Common card_ids:", common_card_ids)
else:
    print("No common card_ids found.")

Number of common card_ids: 123623
Common card_ids: {'C_ID_426c44a6d7', 'C_ID_949aaf2997', 'C_ID_b19904ca49', 'C_ID_78a33e33b0', 'C_ID_c434fc70af', 'C_ID_e9bbb4521f', 'C_ID_142ebf3e73', 'C_ID_0fc5e4991a', 'C_ID_d6a698cdb5', 'C_ID_9e0e7d7e0b', 'C_ID_fe65673a57', 'C_ID_07272cd40c', 'C_ID_14328e5b55', 'C_ID_845fc9605a', 'C_ID_366fa1d28a', 'C_ID_ca57707d2b', 'C_ID_437017458c', 'C_ID_4920eca6bd', 'C_ID_57a580609b', 'C_ID_7a5033fb7f', 'C_ID_c81515bc85', 'C_ID_82a2752ecf', 'C_ID_b5774e62cc', 'C_ID_9260e56be2', 'C_ID_a719ffa1e1', 'C_ID_de7613a15a', 'C_ID_4f80334a64', 'C_ID_9b3c2f6e7a', 'C_ID_4b6a6a9cd3', 'C_ID_06b9ad60dc', 'C_ID_6522481ede', 'C_ID_d95828b063', 'C_ID_a2f5939d8f', 'C_ID_7e0d74aad5', 'C_ID_3a954d1c23', 'C_ID_8452299216', 'C_ID_6f1a35375e', 'C_ID_60636fbfdd', 'C_ID_cbad1cfdf4', 'C_ID_bd220d1d87', 'C_ID_81ed9b1c74', 'C_ID_e34f51d149', 'C_ID_d9e96be3b0', 'C_ID_f04951e9f1', 'C_ID_168f9bff92', 'C_ID_19a77a8953', 'C_ID_2f16781a9d', 'C_ID_9b2c7a11ed', 'C_ID_d0ffefe15b', 'C_ID_a2e85b8bc1'

In [5]:
train = train.merge(features, on='card_id', how='left')
train.fillna(0, inplace=True)

In [6]:
train.drop(columns=['first_active_month'], inplace=True)

In [7]:
train.describe()

,feature_1,feature_2,feature_3,target,total_amount,mean_amount,transaction_count,recency_days,day_mean,day_std,...,category_1_mode,category_1_mode_freq,category_1_mode_ratio,category_2_mode,category_2_mode_freq,category_2_mode_ratio,mean_installments,installment_transactions,authorized_count,not_authorized_count
count,201917.000000,201917.000000,201917.000000,201917.000000,2.019170e+05,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,...,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000,201917.000000
mean,3.105311,1.745410,0.565569,-0.393636,1.109021e+01,1.506766,95.334687,56.994790,15.867783,8.287725,...,0.057326,90.261692,0.935066,2.056914,84.195536,0.872665,0.844793,6.126839,87.599509,7.735178
std,1.186160,0.751362,0.495683,3.850500,1.344916e+04,704.720150,106.910822,76.068227,2.592077,1.429384,...,0.232464,103.874502,0.114399,1.583790,97.844105,0.144281,1.069492,10.991071,101.631691,11.893971
min,1.000000,1.000000,0.000000,-33.219281,-2.089630e+03,-0.745371,2.000000,0.000000,1.000000,0.000000,...,0.000000,2.000000,0.500000,0.000000,2.000000,0.261194,-0.428571,0.000000,2.000000,0.000000
25%,2.000000,1.000000,0.000000,-0.883110,-7.014302e+01,-0.677754,31.000000,7.000000,14.515152,7.645957,...,0.000000,28.000000,0.923077,1.000000,26.000000,0.802469,0.001890,0.000000,27.000000,1.000000
50%,3.000000,2.000000,1.000000,-0.023437,-3.368618e+01,-0.620297,60.000000,27.000000,15.927632,8.458634,...,0.000000,56.000000,1.000000,1.000000,52.000000,0.926829,1.000000,1.000000,54.000000,4.000000
75%,4.000000,2.000000,1.000000,0.765453,-1.471600e+01,-0.502944,118.000000,71.000000,17.317073,9.089856,...,0.000000,112.000000,1.000000,3.000000,104.000000,0.990698,1.334572,8.000000,108.000000,10.000000
max,5.000000,3.000000,1.000000,17.965068,6.010591e+06,316346.913213,2915.000000,450.000000,30.333333,16.970563,...,1.000000,2915.000000,1.000000,5.000000,2554.000000,1.000000,61.294118,518.000000,2540.000000,1510.000000


### LightGBM with Hyperparameter Tuning using optuna

In [ ]:
## Final LightGBM

X = train.drop(columns=['card_id','target'])
y = train['target']

# Define RMSE Function
def rmse(y_true, y_pred):
    return root_mean_squared_error(y_true, y_pred)

# Define Objective Function for Optuna
def objective(trial):
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 10, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 10),
        "metric": "rmse",
        "objective": "regression",
        "random_state": 42
    }

    # 10-Fold Cross Validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    rmse_list = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
        
        y_pred = model.predict(X_val)
        rmse_list.append(rmse(y_val, y_pred))

    return np.mean(rmse_list)  # Return the average RMSE across folds

# **Run Optuna Optimization**
study = optuna.create_study(direction="minimize")  # We want to minimize RMSE
study.optimize(objective, n_trials=20, n_jobs=-1)  # Run 50 trials in parallel

# **Best Hyperparameters & Final Model**
best_params = study.best_params
print("\nBest Hyperparameters:", best_params)

# Train Final Model with Best Parameters
final_lightgbm_model = lgb.LGBMRegressor(**best_params)
final_lightgbm_model.fit(X, y)

# Make Predictions
y_pred = final_lightgbm_model.predict(X)
final_rmse = rmse(y, y_pred)
print("Final RMSE with Best Hyperparameters:", final_rmse)

[I 2025-03-28 10:02:54,486] A new study created in memory with name: no-name-61842bfd-74d6-41a0-814f-0733f322e639
[I 2025-03-28 10:04:18,415] Trial 2 finished with value: 3.7676482518327665 and parameters: {'num_leaves': 32, 'max_depth': 14, 'learning_rate': 0.03603920826533825, 'n_estimators': 189, 'subsample': 0.6806948542445017, 'colsample_bytree': 0.7891886586830905, 'reg_alpha': 0.5440839433515878, 'reg_lambda': 0.9038504928908742}. Best is trial 2 with value: 3.7676482518327665.
[I 2025-03-28 10:04:21,591] Trial 3 finished with value: 3.7637916536910168 and parameters: {'num_leaves': 153, 'max_depth': 4, 'learning_rate': 0.03261382062906748, 'n_estimators': 405, 'subsample': 0.9365415445970839, 'colsample_bytree': 0.6203360949599604, 'reg_alpha': 0.7138954369146628, 'reg_lambda': 0.5919830399874004}. Best is trial 3 with value: 3.7637916536910168.
[I 2025-03-28 10:04:29,982] Trial 4 finished with value: 3.774705657406198 and parameters: {'num_leaves': 128, 'max_depth': 4, 'learni


Best Hyperparameters: {'num_leaves': 153, 'max_depth': 4, 'learning_rate': 0.03261382062906748, 'n_estimators': 405, 'subsample': 0.9365415445970839, 'colsample_bytree': 0.6203360949599604, 'reg_alpha': 0.7138954369146628, 'reg_lambda': 0.5919830399874004}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4404
[LightGBM] [Info] Number of data points in the train set: 201917, number of used features: 23
[LightGBM] [Info] Start training from score -0.393636
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

> Best RMSE: 3.715

> Best Hyperparameters: {'num_leaves': 153, 'max_depth': 4, 'learning_rate': 0.03261382062906748, 'n_estimators': 405, 'subsample': 0.9365415445970839, 'colsample_bytree': 0.6203360949599604, 'reg_alpha': 0.7138954369146628, 'reg_lambda': 0.5919830399874004}

In [ ]:
# Save model
joblib.dump(final_lightgbm_model, 'pareena_lightgbm_model.pkl')


['pareena_lightgbm_model.pkl']

In [ ]:
# Loading model
saved_model = joblib.load('pareena_lightgbm_model.pkl')

In [ ]:
# Prep Test set
X_test = test.merge(features, on='card_id', how='left')
X_test.drop(columns=["first_active_month", "card_id"], inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
# Testing model 
y_pred = saved_model.predict(X_test)

In [35]:
# Converting output to dataframe
test_df = pd.DataFrame(y_pred, columns=["target"])

# Save to CSV with headers
test_df.to_csv("pareena_output.csv", index=False)